# Neighborhoods in Toronto
[3. Explore and cluster]

---

## 1. Intro

This is a Notebook for capstone **Peer-graded Assignment** <br> 
The goal of this notebook is to explore and cluster the neighbourhoods in Toronto

## 2. Explore venues of each neighborhood in Toronto

In [23]:
import pandas as pd
import numpy as np
import json
import requests
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

Load the neighbourhood data

In [16]:
df=pd.read_csv('ZIP_GEO.csv')

In [17]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


Check neighbourhoods on map

In [8]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [128]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

Search venues from Foursquare

In [140]:
CLIENT_ID = 'QFBBKPUZTAQQLUXMEXZ0KOGKUEZEB0UUCYVRGO51PYFAALV2' 
CLIENT_SECRET = 'B0BDIOOTLSC151TYB2KDLHFLTHM03V2OOFF1BUBSWZHVY4RB' 
VERSION = '20180605'

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=radius):
    
    venues_list=[]
    x=1
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(x,': ',name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        x=x+1

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

1 :  Rouge,Malvern
2 :  Highland Creek,Rouge Hill,Port Union
3 :  Guildwood,Morningside,West Hill
4 :  Woburn
5 :  Cedarbrae
6 :  Scarborough Village
7 :  East Birchmount Park,Ionview,Kennedy Park
8 :  Clairlea,Golden Mile,Oakridge
9 :  Cliffcrest,Cliffside,Scarborough Village West
10 :  Birch Cliff,Cliffside West
11 :  Dorset Park,Scarborough Town Centre,Wexford Heights
12 :  Maryvale,Wexford
13 :  Agincourt
14 :  Clarks Corners,Sullivan,Tam O'Shanter
15 :  Agincourt North,L'Amoreaux East,Milliken,Steeles East
16 :  L'Amoreaux West
17 :  Upper Rouge
18 :  Hillcrest Village
19 :  Fairview,Henry Farm,Oriole
20 :  Bayview Village
21 :  Silver Hills,York Mills
22 :  Newtonbrook,Willowdale
23 :  Willowdale South
24 :  York Mills West
25 :  Willowdale West
26 :  Parkwoods
27 :  Don Mills North
28 :  Flemingdon Park,Don Mills South
29 :  Bathurst Manor,Downsview North,Wilson Heights
30 :  Northwood Park,York University
31 :  CFB Toronto,Downsview East
32 :  Downsview West
33 :  Downsview Cen

In [27]:
toronto_venues.shape

(2253, 7)

In [28]:
toronto_venues.head(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [38]:
toronto_venues.rename(columns = {'Neighborhood':'Neighbourhood'}, inplace = True)

check the number of venues for each neighbourhood

In [97]:
vn_num=toronto_venues[['Neighbourhood','Venue']].groupby(by = 'Neighbourhood').count()

In [98]:
vn_num.reset_index(inplace = True)

In [99]:
vn_num=pd.merge(df[['Neighbourhood', 'Latitude', 'Longitude']],vn_num, how = 'left', on = 'Neighbourhood')

In [47]:
vn_num.describe()

,Latitude,Longitude,Venue
count,103.000000,103.000000,100.000000
mean,43.704608,-79.397153,22.530000
std,0.052463,0.097146,29.770318
min,43.602414,-79.615819,1.000000
25%,43.660567,-79.464763,4.000000
50%,43.696948,-79.388790,8.500000
75%,43.745320,-79.340923,23.250000
max,43.836125,-79.160497,100.000000


The average number of venues is 22.53, half of neighbourhoods have less than 8.5 venues

Restructure the venue data

In [73]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
toronto_onehot.shape

(2253, 278)

In [76]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.shape

(100, 278)

Rank the venues

In [94]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [96]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant
1,Agincourt,Breakfast Spot,Lounge,Skating Rink
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Electronics Store
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Sandwich Place
4,"Alderwood,Long Branch",Pizza Place,Pharmacy,Gym


Cluster the neighbourhoods

In [119]:
# set number of clusters
kclusters = 3

toronto_grouped_cl = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_cl)



In [120]:
kmeans.labels_.shape

(100,)

Merge the dataframe

In [121]:
Nb_cl = pd.DataFrame({'Neighbourhood':toronto_venues['Neighbourhood'].unique()})
Nb_cl['Cluster']=kmeans.labels_
Nb_cl=pd.merge(Nb_cl,df[['Neighbourhood', 'Latitude', 'Longitude']],how = 'left', on = 'Neighbourhood')
Nb_cl=pd.merge(Nb_cl,vn_num[['Neighbourhood','Venue']],how = 'left', on = 'Neighbourhood')
Nb_cl=pd.merge(Nb_cl,neighborhoods_venues_sorted,how = 'left', on = 'Neighbourhood')

In [122]:
Nb_cl.head()

,Neighbourhood,Cluster,Latitude,Longitude,Venue,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"Rouge,Malvern",1,43.806686,-79.194353,1.0,Fast Food Restaurant,Yoga Studio,Electronics Store
1,"Highland Creek,Rouge Hill,Port Union",1,43.784535,-79.160497,1.0,Bar,Yoga Studio,Empanada Restaurant
2,"Guildwood,Morningside,West Hill",0,43.763573,-79.188711,7.0,Pizza Place,Rental Car Location,Medical Center
3,Woburn,1,43.770992,-79.216917,4.0,Coffee Shop,Soccer Field,Korean Restaurant
4,Cedarbrae,1,43.773136,-79.239476,7.0,Caribbean Restaurant,Bank,Hakka Restaurant


In [123]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


In [124]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Nb_cl['Latitude'], Nb_cl['Longitude'], Nb_cl['Neighbourhood'], Nb_cl['Cluster']):
    label = folium.Popup(str(poi) + ': Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [125]:
Nb_cl.columns

Index(['Neighbourhood', 'Cluster', 'Latitude', 'Longitude', 'Venue',
       '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue'],
      dtype='object')

In [126]:
Nb_cl[['Neighbourhood', 'Cluster']].groupby(by = 'Cluster').count()

,Neighbourhood
Cluster,
0,15
1,83
2,2


Most neighbourhoods are in cluster 1

In [127]:
Nb_cl[Nb_cl.Cluster==1]

,Neighbourhood,Cluster,Latitude,Longitude,Venue,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,"Rouge,Malvern",1,43.806686,-79.194353,1.0,Fast Food Restaurant,Yoga Studio,Electronics Store
1,"Highland Creek,Rouge Hill,Port Union",1,43.784535,-79.160497,1.0,Bar,Yoga Studio,Empanada Restaurant
3,Woburn,1,43.770992,-79.216917,4.0,Coffee Shop,Soccer Field,Korean Restaurant
4,Cedarbrae,1,43.773136,-79.239476,7.0,Caribbean Restaurant,Bank,Hakka Restaurant
5,Scarborough Village,1,43.744734,-79.239476,2.0,Playground,Smoke Shop,Electronics Store
6,"East Birchmount Park,Ionview,Kennedy Park",1,43.727929,-79.262029,4.0,Discount Store,Convenience Store,Department Store
7,"Clairlea,Golden Mile,Oakridge",1,43.711112,-79.284577,10.0,Bakery,Bus Line,Metro Station
8,"Cliffcrest,Cliffside,Scarborough Village West",1,43.716316,-79.239476,2.0,American Restaurant,Motel,Dive Bar
9,"Birch Cliff,Cliffside West",1,43.692657,-79.264848,4.0,Café,College Stadium,General Entertainment
10,"Dorset Park,Scarborough Town Centre,Wexford He...",1,43.757410,-79.273304,7.0,Indian Restaurant,Furniture / Home Store,Pet Store
